In [1]:
import pandas as pd
#from ddbapi import zp_issues, zp_pages, list_column, filter
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import re
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import swifter
import time
import glob
import os
import PIL
import shutil
from PIL import ImageFile
from torch import nn
from datetime import datetime
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [82]:
hermes_df = pd.read_pickle("newspapers_ger_1914_part_1")

In [65]:
hermes_df.head()

,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,plainpagefulltext
0,222VFGDROAVP6UQSQML7Q7OERUCLJPRM-ALTO3061500_D...,1,Bensberger Volkszeitung. 1907-1929,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2706805-5,1914-02-07 12:00:00,"[Bensberg, Bergisch Gladbach, Bergisch Gladbac...",[ger],af851bb9-e1b0-4cfe-bb8c-34a441887288,[/data/altos/22/2V/222VFGDROAVP6UQSQML7Q7OERUC...,ALTO3061500_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,„ 2666000000000000000OOO e Erscheint 6 * Diens...
1,222VFGDROAVP6UQSQML7Q7OERUCLJPRM-ALTO3061501_D...,2,Bensberger Volkszeitung. 1907-1929,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2706805-5,1914-02-07 12:00:00,"[Bensberg, Bergisch Gladbach, Bergisch Gladbac...",[ger],af851bb9-e1b0-4cfe-bb8c-34a441887288,[/data/altos/22/2V/222VFGDROAVP6UQSQML7Q7OERUC...,ALTO3061501_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,Die Vollendung der Tanganjika - Bahn Wie amtli...
2,222VFGDROAVP6UQSQML7Q7OERUCLJPRM-ALTO3061502_D...,3,Bensberger Volkszeitung. 1907-1929,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2706805-5,1914-02-07 12:00:00,"[Bensberg, Bergisch Gladbach, Bergisch Gladbac...",[ger],af851bb9-e1b0-4cfe-bb8c-34a441887288,[/data/altos/22/2V/222VFGDROAVP6UQSQML7Q7OERUC...,ALTO3061502_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,Mitarbeiter und Assistenten Dr . M . Krause in...
3,222VFGDROAVP6UQSQML7Q7OERUCLJPRM-ALTO3061503_D...,4,Bensberger Volkszeitung. 1907-1929,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2706805-5,1914-02-07 12:00:00,"[Bensberg, Bergisch Gladbach, Bergisch Gladbac...",[ger],af851bb9-e1b0-4cfe-bb8c-34a441887288,[/data/altos/22/2V/222VFGDROAVP6UQSQML7Q7OERUC...,ALTO3061503_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Schleimat ; Ratorh , Ver schleimung , Krampf =..."
4,226ABITF5AIJVEEYKQLH4ZG5S2PNHO3R-ALTO3782620_D...,1,Der Volksfreund : Tageszeitung für das werktät...,INLVDM4I3AMZLTG6AE6C5GZRJKGOF75K,Badische Landesbibliothek,2871891-4,1914-03-04 12:00:00,"[Karlsruhe, Offenburg]",[ger],256ec72e-9007-4340-9afd-f29a7db9d779,[/data/altos/22/6A/226ABITF5AIJVEEYKQLH4ZG5S2P...,ALTO3782620_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"tio . 63 . Karlsruhe , Mittwoch den 4 * Marz 1..."


In [4]:
unique_id_df = hermes_df['page_id'].str.extract(r'\A([^-]+)').value_counts().rename_axis('page_id').reset_index(name='counts')

# Get Hermes Images

In [7]:
def get_page_urls(page_id):
    api_url = f'https://api.deutsche-digitale-bibliothek.de/items/{page_id}?oauth_consumer_key=jCdsZUOrAMOHdtPHvh8SbRK2nuDuVmzp4GAZxTGll0CgjALri7R1725871456006'
    try:
        soup = BeautifulSoup(requests.get(api_url).json()['source']['record']['$'], features="xml")
        if soup.find_all('mets:file', {"MIMETYPE": "image/jpeg"}):
            link_list = [elem.find('mets:FLocat').get('xlink:href') for elem in soup.find_all('mets:file', {"MIMETYPE": "image/jpeg"})]
        elif soup.find_all('mets:file', {"MIMETYPE": "image/jpg"}):
            link_list = [elem.find('mets:FLocat').get('xlink:href') for elem in soup.find_all('mets:file', {"MIMETYPE": "image/jpg"})]
    except NameError:
        link_list = []
        print(f'No urls found at ID: {page_id}')
    except KeyError:
        link_list = []
        print(f'No urls found at ID: {page_id}')
    return link_list

In [15]:
image_urls = []
for index, row in tqdm(unique_id_df.iterrows(), total=unique_id_df.shape[0]):
    image_list = get_page_urls(row['page_id'])
    image_urls.append(image_list)
    if index % 500 == 0:
        time.sleep(1)

  0%|          | 0/14193 [00:00<?, ?it/s]

No urls found at ID: AYXZCXB3A75ERP66AAQ2LTLLG5VDDSRX
No urls found at ID: 76JFHCRGVO7RL7WM7WDPADBO7QTGNZJS
No urls found at ID: BMVKVGEVMQAXCE2WNJ4TFS6FA2T34LIN
No urls found at ID: MZF2AO4R6CFTNT6VCVBBOH4FRMCRFPRR
No urls found at ID: ZT37FARLD3UPIP6XMKJB5DM4G6RQDIX2
No urls found at ID: J6GTXXEMF53EICUXX4R4YMHJWSVWFKSR
No urls found at ID: NKZAF3RYYDR6NU2WB2K5GZI33HYRE326
No urls found at ID: WSUETVBJ3C5D22726SDW2TNXEAEBSYMC
No urls found at ID: BWMSSNJJIUR4LGLMMTRM2HUW2IRJJUOD
No urls found at ID: PKTIAALVJP7EQXYAMCU32NTXGV4RPXIK
No urls found at ID: MGW4T7VHKEIWN7EJ5Q2RZ73GOUGEJ26R
No urls found at ID: PZXNGXYLQPX3EZVDHC5R3OHUNZAMMAPY
No urls found at ID: YQQ6WB3YR226NGPMF3AD3D2D2K6SXRGN
No urls found at ID: 4SUER6ECELSVSA3S5EK6ZN5JYCGZCKX3
No urls found at ID: S4KUQCMX5CIYLRSVDOQBAOP3EEJZBFZD
No urls found at ID: 5VBO6AFXWZLTLWCTC7JM2HJBRDESBZJM
No urls found at ID: QIVBGFHCR3NKSQRBACURYECTYPUGRZ77
No urls found at ID: KWCAP6VVVUBR4T2SYJDVAEBETWJNFG7M
No urls found at ID: SZZ4H6T

In [17]:
unique_id_df['image_url'] = image_urls

In [99]:
reduced_urls = []
for index, row in tqdm(unique_id_df.iterrows(), total=unique_id_df.shape[0]):
    reduced_list = [x for x in row['image_url'] if 'thumb' not in x]
    reduced_list = [x for x in reduced_list if 'small' not in x]
    reduced_list = reduced_list[-row['counts']:]
    reduced_urls.append(reduced_list)

  0%|          | 0/14193 [00:00<?, ?it/s]

In [100]:
unique_id_df['reduced_url'] = reduced_urls

In [123]:
unique_id_df.to_pickle("img_newspapers_ger_1914_part_1.pkl") 

In [2]:
unique_id_df = pd.read_pickle("img_newspapers_ger_1914_part_1.pkl")

In [3]:
img_folder = 'N:\\Jobads\\hermes_images'

In [4]:
unique_id_df = unique_id_df[unique_id_df['reduced_url'].apply(len) != 0]

In [28]:
for index, row in tqdm(unique_id_df.iloc[1601:1602].iterrows(), total=unique_id_df[1283:].shape[0]):
    if index % 100 == 0:
        time.sleep(0.5)
    image_id = row['page_id']
    for page, image_url in enumerate(tqdm(row['reduced_url'])):
        if page % 10 == 0:
            time.sleep(0.5)
        image_type = requests.head(image_url)
        if 'image' in image_type.headers.get('content-type'):
            image_ext = re.search(r'/(\S+)', image_type.headers.get('content-type')).group(1)
            os.makedirs(f'{img_folder}\\{image_id}', exist_ok=True)
            image_name = f'{img_folder}\\{image_id}\\{image_id}_{page}.{image_ext}'
            image = requests.get(image_url)
            with open(image_name, 'wb') as file:
                file.write(image.content)

        else:
            print(f'No image found for {image_id} page {page}')

  0%|          | 0/12390 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#No image found for ENJ7TMO7VCIPRZUGAZYWFTYJXCRXBWEC page 25
#No image found for 672MS4S6U4QKGKWSD24J6BNFVH2IA24R page 14
#No image found for L7FWGLRG5SPKI2CXIAAETY3N5KFLBLCH page 4
#No image found for ZLGQUCP476IHTL24ZD77ZV2YFZT7Q4ZQ page 19
#No image found for B4ZLKXPRT34TEDJK5CP2LTJY2OPOIJWN page 0 - 14
#No image found for D7BXDCE4EBFLPA4VLGHXG4HTAAB2FATD page 1 - 3
#No image found for MFQHU7SZ2QAZVSLT6SE4V7I2RWIAAJWV page 0 - 1
#No image found for 2XXGHUL7BGCBBDTWKTKW63TNPPUXDTNU page 7
#No image found for GWGVUQ6ENC3WQFPTIYEHCK44BUDJYIW5 page 3

In [27]:
row['reduced_url'][:4]

['https://zeitpunkt.nrw/ulbbn/download/webcache/0/5894173',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/5894174',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/5894175',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/5894176']

In [21]:
unique_id_df.iloc[1601:1602]

,page_id,counts,image_url,reduced_url
1602,D7BXDCE4EBFLPA4VLGHXG4HTAAB2FATD,16,[https://zeitpunkt.nrw/ulbbn/download/webcache...,[https://zeitpunkt.nrw/ulbbn/download/webcache...


In [32]:
unique_id_df[unique_id_df['page_id'] == 'GWGVUQ6ENC3WQFPTIYEHCK44BUDJYIW5']['reduced_url'].iloc[0]

['https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739421',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739422',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739423',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739424',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739425',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739426',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739427',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739428',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739429',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739430',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739431',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739432',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739433',
 'https://zeitpunkt.nrw/ulbbn/download/webcache/0/6739434']

In [49]:
test = requests.get(image_url)

In [185]:
test = requests.head(unique_id_df['reduced_url'].iloc[0][0])

In [186]:
re.search(r'/(\S+)', test.headers.get('content-type')).group(1)

'jpeg'

In [61]:
with open(f'{img_folder}\\t\\{image_id}_{page}.{image_ext}', 'wb') as file:
    file.write(test.content)

# Identify Pages containing Job Ads

In [33]:
files_img = glob.glob(f'{img_folder}/**/*.*', recursive=True)

In [34]:
cloud_df = pd.DataFrame({'image_path': files_img})

In [28]:
cloud_df['page_id'] = cloud_df['image_path'].str.extract(r'\\([\dA-Z]{2,})_')

In [36]:
# with open("hermes_ads.txt", "w") as file:
#     file.write("")

In [38]:
from transformers import AutoModelForImageClassification, AutoImageProcessor
import torch

repo_name = "DiT_classification_jobads_finetuned"

image_processor = AutoImageProcessor.from_pretrained(repo_name)
model = AutoModelForImageClassification.from_pretrained(repo_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BeitForImageClassification(
  (beit): BeitModel(
    (embeddings): BeitEmbeddings(
      (patch_embeddings): BeitPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BeitEncoder(
      (layer): ModuleList(
        (0): BeitLayer(
          (attention): BeitAttention(
            (attention): BeitSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=False)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BeitSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): BeitIntermediate(
            (dense): Linear(

In [59]:
f = open("hermes_ads.txt", "r")
data = f.read() 
data_into_list = data.split("\n")
all_ads = [x.split(' ')[0] for x in data_into_list]

In [60]:
len(list(set(pages) - set(all_ads)))

0

In [58]:
pages = cloud_df['image_path'].to_list()
pages = list(set(pages) - set(all_ads))
n = 100
df_batches = [pages[i:i+n] for i in range(0,len(pages),n)]
back_up = 0
for batch in tqdm(df_batches):
    images = []
    preds = []
    probs = []
    restart = False
    retry = 0
    for image in batch:
        while True:
            try:
                img_data = PIL.Image.open(image).convert("RGB")
                images.append(img_data)
                if restart:
                    t = time.localtime()
                    current_time = time.strftime("%H:%M:%S", t)
                    restart = False
                    retry = 0
                    print(f'{current_time}: Reconnected')
            except OSError as e1:
                t = time.localtime()
                current_time = time.strftime("%H:%M:%S", t)
                print(f'{current_time}: {e1} \nRate limit. Retry after 1 minute')
                time.sleep(60)
                restart = True
                if retry == 3:
                    print(f'Tried {retry} times. Skipping image')
                    break
                retry += 1
                continue
            else:
                break
            
    encoding = image_processor(images, return_tensors="pt")
    encoding = encoding.to(device)
    
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits
    preds.extend(logits.argmax(-1).tolist())
    probs.extend(nn.functional.softmax(outputs.logits, dim=-1).amax(-1).tolist())
    with open("hermes_ads.txt", "a") as f:
        for (image, pred, prob) in zip(batch, preds, probs):
            f.write("{0} {1} {2}\n".format(image, pred, prob))
    del encoding
    torch.cuda.empty_cache()
    back_up += 1
    if back_up == 500:
        now = datetime.now()
        dest = 'Backup\\''hermes_ads'+'_'+now.strftime("%d_%m_%Y_%H_%M")+'.txt'
        shutil.copy('hermes_ads.txt', dest)
        back_up = 0

  0%|          | 0/88 [00:00<?, ?it/s]

09:13:12: [Errno 22] Invalid argument: 'N:\\Jobads\\hermes_images\\GO7FSRIO6VBUHRIADAULQADLXCJMJRA2\\GO7FSRIO6VBUHRIADAULQADLXCJMJRA2_9.jpeg' 
Rate limit. Retry after 1 minute
09:14:13: Reconnected


In [83]:
hermes_df['paper_id'] = hermes_df['page_id'].str.extract(r'\A([^-]+)')
hermes_df['join_id'] = hermes_df['paper_id'] + ' ' + hermes_df['pagenumber'].astype(str)

In [728]:
hermes_res_df = pd.read_csv('hermes_ads.txt', sep=" ", header=None)
hermes_res_df.rename(columns={0: "file_path", 1: "classification", 2: "probability"}, inplace = True)

In [86]:
hermes_res_df['paper_id'] = hermes_res_df['file_path'].str.extract(r'\\([^\\]+)_\d+\.')
hermes_res_df['page_number'] = hermes_res_df['file_path'].str.extract(r'(\d+)\.')
hermes_res_df['page_number'] = pd.to_numeric(hermes_res_df['page_number'])
hermes_res_df['page_number'] = hermes_res_df['page_number']+1
hermes_res_df['join_id'] = hermes_res_df['paper_id'] + ' ' + hermes_res_df['page_number'].astype(str)

In [103]:
hermes_ads_df = hermes_df.join(hermes_res_df[["file_path", "classification", "probability", 'join_id']].set_index('join_id'), on='join_id')
hermes_ads_df = hermes_ads_df.dropna()
hermes_ads_df = hermes_ads_df[hermes_ads_df['classification'] == 1]
hermes_ads_df.drop(columns=['join_id'], inplace = True)

In [119]:
hermes_ads_df.to_csv('newspapers_ger_1914_part_1_job_ads.csv', index=False)

In [3]:
hermes_ads_df = pd.read_csv('newspapers_ger_1914_part_1_job_ads.csv') 

In [730]:
hermes_ads_df = hermes_res_df[hermes_res_df['classification'] == 0]

In [731]:
hermes_ads_df['true_ad'] = None

C:\Users\Raven Adam\AppData\Local\Temp\ipykernel_20776\1867748654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hermes_ads_df['true_ad'] = None


In [963]:
i = 0

In [ ]:
PIL.Image.open(hermes_ads_df.iloc[i]['file_path'])

In [953]:
hermes_ads_df['true_ad'].iloc[i] = 1
i += 1

C:\Users\Raven Adam\AppData\Local\Temp\ipykernel_20776\3372319309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hermes_ads_df['true_ad'].iloc[i] = 1


In [965]:
hermes_ads_df['true_ad'].iloc[i] = 0
i += 1

C:\Users\Raven Adam\AppData\Local\Temp\ipykernel_20776\988249202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hermes_ads_df['true_ad'].iloc[i] = 0


In [968]:
len(hermes_ads_df[hermes_ads_df['true_ad']==0])

101

In [969]:
hermes_ads_df['true_ad']#.iloc[i-3:i+3]

0           0
1           0
2           0
3           0
4           0
         ... 
58777    None
58778    None
58780    None
58784    None
58785    None
Name: true_ad, Length: 34179, dtype: object

In [972]:
hermes_ads_df[hermes_ads_df['true_ad']==0].to_csv('newspapers_ger_1914_part_1_no_ads.csv', index=False)

In [668]:
hermes_ads_df.iloc[i]['file_path']

'N:\\Jobads\\hermes_images\\2\\2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH_29.jpeg'

# OCR Post-Correction

In [976]:
def get_byt_len(text):
    return len(tokenizer(text, return_tensors="pt").input_ids[0])

In [977]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("hmbyt_ONB_150")

In [973]:
hermes_ads_df = pd.read_csv('classified_ads.csv')

In [981]:
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'ﬁ', r'fi', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'ﬂ', r'fl', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'¬', r'-', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'¶', r'', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'˖', r'+', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'‟', r'"', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'⅙', r'1/6', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'ﬀ', r'ff', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'ꝓ', r'ff', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'ů', r'u', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'̃', r'', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'̈', r'', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'ﬄ', r'ffl', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'⸗', r'-', regex=True)
hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(r'ꝛ', r'r', regex=True)
for vok, uml in [('a', 'ä'), ('o', 'ö'), ('u', 'ü')]:
    hermes_ads_df['text'] = hermes_ads_df['text'].str.replace(vok+r'ͤ', uml, regex=True)

In [1077]:
hermes_ads_df['byt_len'] = hermes_ads_df['text'].apply(get_byt_len)
hermes_ads_df['uniq_id'] = hermes_ads_df.index

In [1080]:
text_list = []
byt_len_list = []
for index, row in tqdm(hermes_ads_df.iterrows(), total=hermes_ads_df.shape[0]):
    if row['byt_len'] > 150:
        text_split = row['text'].split()
        split_idx = round(len(text_split)/2)
        
        text_half_1 = text_split[:split_idx]
        text_half_1 = " ".join(text_half_1)
        text_half_2 = text_split[split_idx:]
        text_half_2 = " ".join(text_half_2)
        new_byt_len = [get_byt_len(x) for x in [text_half_1, text_half_2]]

        text_list.append([text_half_1, text_half_2])
        byt_len_list.append(new_byt_len)

    else:
        text_list.append(row['text'])
        byt_len_list.append(row['byt_len'])

  0%|          | 0/1021 [00:00<?, ?it/s]

In [1081]:
split_df = pd.DataFrame({'uniq_id': hermes_ads_df['uniq_id'], 'text': text_list, 'byt_len': byt_len_list})
split_df = split_df.explode(['text', 'byt_len']).reset_index(drop=True)

In [1103]:
text_list = []
byt_len_list = []
for index, row in tqdm(split_df.iterrows(), total=split_df.shape[0]):
    if row['byt_len'] > 150:
        text_split = row['text'].split()
        split_idx = round(len(text_split)/2)
        
        text_half_1 = text_split[:split_idx]
        text_half_1 = " ".join(text_half_1)
        text_half_2 = text_split[split_idx:]
        text_half_2 = " ".join(text_half_2)
        new_byt_len = [get_byt_len(x) for x in [text_half_1, text_half_2]]

        text_list.append([text_half_1, text_half_2])
        byt_len_list.append(new_byt_len)

    else:
        text_list.append(row['text'])
        byt_len_list.append(row['byt_len'])

  0%|          | 0/5038 [00:00<?, ?it/s]

In [1104]:
split_df = pd.DataFrame({'uniq_id': split_df['uniq_id'], 'text': text_list, 'byt_len': byt_len_list})
split_df = split_df.explode(['text', 'byt_len']).reset_index(drop=True)

In [1105]:
split_df.sort_values('byt_len')

,uniq_id,text,byt_len
3681,724,WM,3
316,58,an,3
306,50,ala,4
4538,817,285,4
3478,619,ME:,4
...,...,...,...
2451,387,In Eine Vereinigung der Drahtindustrie sucht f...,150
1688,383,". vertr . , bew . in ſäm J usſtatt . ſof . ſp ...",150
4367,816,"Off . Martha Rothe , ſlichiadterſtraße 21 . i ...",150
1084,379,n für Sangatorium im inland gegen hoh . Lohn O...,150


In [1035]:
model = AutoModelForSeq2SeqLM.from_pretrained("hmbyt_ONB_150_no_overlap.model").to("cuda")

In [1046]:
preds = split_df['text'].to_list()
n = 80
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
tokenizer.padding_side = "left"
pred_chunk = [preds[i:i + n] for i in range(0, len(preds), n)]
res = []

for chunk in tqdm(pred_chunk):
    input = chunk
    input = tokenizer.batch_encode_plus(input, return_tensors="pt", padding=True)#.input_ids
    input = input.to("cuda")
    output = model.generate(**input, max_new_tokens=len(input.input_ids[0])+5, num_beams=4, do_sample=True)
    temp_res = tokenizer.batch_decode(output, skip_special_tokens=True)
    temp_res = [s.replace("<pad>", "") for s in temp_res]
    res.extend(temp_res)
    del input
    torch.cuda.empty_cache()

  0%|          | 0/64 [00:00<?, ?it/s]

In [1048]:
split_df['post_text'] = res

In [1054]:
del model

NameError: name 'model' is not defined

In [1056]:
model = AutoModelForSeq2SeqLM.from_pretrained("hmbyt_pos_pred_150_e15").to("cuda")

In [1055]:
torch.cuda.empty_cache()

In [1058]:
preds = split_df['post_text'].to_list()
n = 80
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id
tokenizer.padding_side = "left"
pred_chunk = [preds[i:i + n] for i in range(0, len(preds), n)]
res = []

for chunk in tqdm(pred_chunk):
    input = chunk
    input = tokenizer.batch_encode_plus(input, return_tensors="pt", padding=True)#.input_ids
    input = input.to("cuda")
    output = model.generate(**input, max_new_tokens=len(input.input_ids[0]), num_beams=5, do_sample=True)
    temp_res = tokenizer.batch_decode(output, skip_special_tokens=True)
    temp_res = [s.replace("<pad>", "") for s in temp_res]
    res.extend(temp_res)
    del input
    torch.cuda.empty_cache()

  0%|          | 0/64 [00:00<?, ?it/s]

In [1059]:
split_df['gen_positions'] = res

In [1115]:
fin_df = split_df.groupby('uniq_id', as_index = False)[['text', 'post_text', 'gen_positions']].agg(' '.join)
fin_df['gen_positions'] = fin_df['gen_positions'].str.replace('none', '')

In [1116]:
fin_df['gen_positions'] = fin_df['gen_positions'].str.replace('none', '')

In [1117]:
fin_df

,uniq_id,text,post_text,gen_positions
0,0,"Portikusstr. 4, II.","Portikusstr. 4, II.",
1,1,llffiriel'8«si8k sucht für einige Tagesstunden...,Friedrich sucht für einige Tagesstunden Be- sc...,<Gesellschafterin> <Begleiterin> <Malerin>
2,2,Juristischer Hilfsarbeiter (Volontär) kann bei...,Juristischer Hilfsarbeiter (Volontär) kann bei...,<Hilfsarbeiter>
3,3,^onaortdiroktion W'. «1vG st'. INLtner).,Annoncendirektion W. Kleiner).,
4,4,EM. M WUSLN Köllig 4odavu8tr.,EM. M. WUSIN Köllig Sodavustr.,
...,...,...,...,...
1016,1016,C^auetoortbolten ober ähnliche»\nV sucht finbe...,Landvortheilen ober ähnliches sucht finderlose...,
1017,1017,"L^aubwart, gelernter Schlosser\n•V und Elektri...","Laubwart, gelernter Schlosser und Elektriker, ...",<Laubwart> <Schlosser> <Elektriker>
1018,1018,fctattfljaUriflcr 13 ue sucht wegen A bbruch» ...,Statthalterstraße 13 gesucht wegen Abbruch der...,
1019,1019,junges Ehepaar sucht Berwalter-\nV holten »um ...,junges Ehepaar sucht Verwalter- holten zum 1. ...,


In [1121]:
hermes_ads_df = hermes_ads_df.join(fin_df.set_index('uniq_id')[['post_text', 'gen_positions']], on='uniq_id')

In [1123]:
hermes_ads_df.drop(columns=['byt_len', 'uniq_id'], inplace=True)

In [1124]:
hermes_ads_df.to_csv('classified_ads_post_corr.csv', index=False)